# 🐾 Cat vs. Dog Image Classifier 🐈

**Goal:** To train a deep learning model that can accurately classify images as either a 'cat' or a 'dog'.

This notebook uses the `fastai` library to build a simple but powerful computer vision model based on the Oxford-IIIT Pet Dataset.



### Workflow:
1.  **Download & Prepare Data:** Get the dataset and set it up for the model.
2.  **Train the Model:** Use a pre-trained Convolutional Neural Network (CNN) and fine-tune it on the downloaded data.
3.  **Export the Model:** Save the trained model to a `.pkl` file for use in a web application.

---

## 1. Install packages and download the dataset

In [ ]:
# STEP 1: Install and import fastai
# The '!' lets us run a command line command.
# '-Uqq' means 'upgrade' and 'quietly' (to keep the output clean)
!pip install -Uqq fastai

# Import all the tools we need from the fastai vision library
from fastai.vision.all import *

In [ ]:
# STEP 2: Download and prepare the dataset
# fastai's 'untar_data' function downloads a dataset, extracts it,
# and returns the path where it's stored.
path = untar_data(URLs.PETS)

---
## 2. Structuring the Data with a `DataBlock`

The `DataBlock` is a core `fastai` concept. A blueprint that describes how to assemble the raw data into something a model can train on. Specify:
- **`blocks`**: What are independent (`ImageBlock`) and dependent (`CategoryBlock`) variables?
- **`get_items`**: Find all our input items? (`get_image_files`)
- **`splitter`**: Split the data into training and validation sets? (`RandomSplitter`)
- **`get_y`**: Get the label for each image? (From the filename, using a regular expression)
- **`item_tfms`**: Apply transforms to each individual item? (`Resize`)
- **`batch_tfms`**: Apply transforms to a whole batch of items at once on the GPU? (`aug_transforms` for data augmentation)

In [ ]:
# STEP 3 (Corrected): Define the DataBlock and create DataLoaders

# This function determines if an image is a cat or a dog.
# We use x.name to get the filename string from the path object.
def is_cat(x): return x.name[0].isupper() # <--- THE FIX IS HERE

dblock = DataBlock(
    blocks=[ImageBlock, CategoryBlock],
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=is_cat,
    item_tfms=[Resize(224)]
)

# Create the DataLoaders object
dls = dblock.dataloaders(path/"images")

In [ ]:
# Verify our data looks correct
dls.show_batch(max_n=6)

---
## 3. Train the Model

Train the model using a powerful technique called **transfer learning**. Instead of teaching a model from scratch, start with one that has already been trained on millions of general images (a dataset called ImageNet) by using the famous **`resnet34`** architecture.

The `vision_learner` function builds the model. Use `error_rate` as the metric to see how well it's performing.

Finally, use the `.fine_tune()` method, which is the best way to train a transfer learning model. It quickly and safely adapts the pre-trained model to the specific cat-vs-dog task.

In [ ]:
# STEP 4: Create and train the vision learner
learn = vision_learner(dls, resnet34, metrics=error_rate)

# Fine-tune the model for one "epoch" (one full pass through the data)
learn.fine_tune(1)

---
## 4. Export the Model for Inference

Now that the model is trained, Save it. Use the `learn.export()` method, which bundles the entire `Learner` object (the model architecture, the trained weights, and the necessary data transformations) into a single file called a `.pkl` file. This file contains everything the final application will need to make predictions.

In [ ]:
# STEP 5: Export the learner object
learn.export('cat_dog_classifier_v1.pkl')